In [1]:
import pandas as pd
import numpy as np
from itertools import combinations
import random
import os
import time
from tqdm import tqdm
import datetime

In [2]:
##Create Lineups

##User defined variables
#num lineups in tournament
num_lineups = 15000

#cumulative ownership for the field
cum_per_owned = 6
start = datetime.datetime.now()


#Functions
def random_combination(iterable, r, player): 
    #Random selection from combinations
    iter_count = 0 
    combo_salary = 0
    salary_ls = []
    while combo_salary < 48800 or combo_salary > 50000:
        iter_count += 1
        if iter_count > 60000:
            if player == 'placeholder':
                print('placeholder error')
                
            else:
                print('combination impossible')
                
            return salary_ls
        
        if r == 5:
            combo_salary = salary_dict[player]
        else:
            combo_salary = 0
        
        pool = tuple(iterable)
        n = len(pool)
        indices = sorted(random.sample(range(n), r))
        team = tuple(pool[i] for i in indices)
        
        for t in team:
            combo_salary += salary_dict[t]
        
        salary_ls.append(tuple((team, combo_salary)))
        
    return tuple(pool[i] for i in indices)


def random_lineup_generator(iterable, r, lineups_to_create, player):
    temp_ls = []
    for i in range(lineups_to_create):
        
        temp_ls.append(random_combination(iterable, r, player))
    
    return temp_ls


def lineup_difference (p, player):
    curr_df_len = len(lineup_df)

    adjust_df_len = num_lineups - curr_df_len

    target_own = pid_dict[p]['own']
    target_lus = round(target_own * num_lineups) + adjust_df_len

    actual_lus = count_df['count'].loc[count_df['name'] == player].values[0]

    lu_diff = int(target_lus - actual_lus)
    
    return lu_diff

##Create return_ownership

def return_ownership(player):
    for key, value in pid_dict.items():
        if value['name'] == player:
            return value['own']

#Read salaries and ownership
player_df = pd.read_csv('/home/valesco/Datasets/PGA_Data/salary_folder/pga_champ_salaries.csv')
player_df = player_df[player_df['owned'] > 0]

players = list(player_df['name'].values)
salary_dict = dict(zip(player_df['name'], player_df['salary']))

pid_dict = {}

for p, player in enumerate(players):
    salary = player_df['salary'].loc[player_df['name'] == player].values[0]
    own = player_df['owned'].loc[player_df['name'] == player].values[0]
    pid_dict[p] = {'name': player, 'salary': salary, 'own': own, 'lineup_diff': 0}

pids = list(pid_dict.keys())

#generate random num lineups for tournament
random_lineups = random_lineup_generator(players, 6, num_lineups, 'player')

In [7]:
lineup_df = pd.DataFrame(random_lineups)
salary_dict = dict(zip(player_df['name'], player_df['salary']))

cols = ['player1', 'player2', 'player3', 'player4', 'player5', 'player6']
lineup_df.columns = cols
for col in cols:
    lineup_df[col + '_sal'] = lineup_df[col].map(lambda x: salary_dict[x])
    
lineup_df['total_salary'] = lineup_df['player1_sal'] + lineup_df['player2_sal'] + lineup_df['player3_sal'] + lineup_df['player4_sal'] + lineup_df['player5_sal'] + lineup_df['player6_sal'] 
    
lineup_df

,player1,player2,player3,player4,player5,player6,player1_sal,player2_sal,player3_sal,player4_sal,player5_sal,player6_sal,total_salary
0,Jon Rahm,Brooks Koepka,Adam Scott,Bubba Watson,Si Woo Kim,Zach Johnson,10100,9200,8600,7300,7100,6800,49100
1,Jordan Spieth,Brooks Koepka,Paul Casey,Webb Simpson,Gary Woodland,Chez Reavie,12000,9200,7800,6800,6700,6600,49100
2,Rory McIlroy,Tommy Fleetwood,Alexander Noren,Daniel Berger,Russell Knox,Nicolas Colsaerts,11800,8000,7900,7700,6900,6500,48800
3,Rory McIlroy,Justin Thomas,Matthew Fitzpatrick,Russell Henley,Ian Poulter,Charles Howell,11800,8900,7400,7400,7000,6400,48900
4,Rory McIlroy,Matt Kuchar,Marc Leishman,Bernd Wiesberger,Kevin Na,Webb Simpson,11800,8800,8100,7000,6900,6800,49400
5,Dustin Johnson,Henrik Stenson,Ryan Moore,Jimmy Walker,Bill Haas,Ryan Fox,11400,9800,7300,6900,6800,6600,48800
6,Dustin Johnson,Justin Rose,Justin Thomas,Jamie Lovemark,Steve Stricker,Jason Dufner,11400,9000,8900,7200,6800,6700,50000
7,Jordan Spieth,Hideki Matsuyama,Bernd Wiesberger,Hideto Tanihara,Ryan Fox,David Lingmerth,12000,10500,7000,6700,6600,6600,49400
8,Hideki Matsuyama,Adam Scott,Phil Mickelson,Marc Leishman,Charley Hoffman,Jim Furyk,10500,8600,8500,8100,7600,6400,49700
9,Jordan Spieth,Jason Day,Bernd Wiesberger,Thorbjorn Olesen,Brandon Stone,Tyrrell Hatton,12000,10400,7000,7000,6800,6600,49800


In [14]:
random_lineups

[('Jason Day',
  'Jon Rahm',
  'Phil Mickelson',
  'Louis Oosthuizen',
  'Adam Hadwin',
  'Anirban Lahiri'),
 ('Rory McIlroy',
  'Rickie Fowler',
  'Robert Streb',
  'Hao-Tong Li',
  'Brandon Stone',
  'Ross Fisher'),
 ('Rory McIlroy',
  'Hideki Matsuyama',
  'Matthew Fitzpatrick',
  'Jimmy Walker',
  'Chez Reavie',
  'Nicolas Colsaerts'),
 ('Rory McIlroy',
  'Jason Day',
  'Bubba Watson',
  'James Hahn',
  'Kelly Kraft',
  'Francesco Molinari'),
 ('Jordan Spieth',
  'Hideki Matsuyama',
  'Tony Finau',
  'Patrick Rodgers',
  'David Toms',
  'Younghan Song'),
 ('Rory McIlroy',
  'Jon Rahm',
  'Russell Henley',
  'Ian Poulter',
  'Robert Streb',
  'Jason Dufner'),
 ('Hideki Matsuyama',
  'Sergio Garcia',
  'Branden Grace',
  'Paul Casey',
  'Byeong-Hun An',
  'Kelly Kraft'),
 ('Jordan Spieth',
  'Rickie Fowler',
  'Wesley Bryan',
  'Byeong-Hun An',
  'Jeunghun Wang',
  'Cody Gribble'),
 ('Jordan Spieth',
  'Jason Day',
  'Peter Uihlein',
  'Jonas Blixt',
  'Ryan Fox',
  'Younghan Song'),

In [4]:
lineup_df = pd.DataFrame(random_lineups)

#Count players in lineups
count_df = lineup_df.apply(pd.Series.value_counts)

#Turn counts into percent
count_df['count'] = count_df.sum(axis = 1)
count_df.reset_index(inplace = True)
count_df.rename(columns = {'index': 'name'}, inplace = True)
count_df['owned'] = count_df['name'].apply(lambda x: return_ownership(x))
count_df['target_lus'] = count_df['owned'] * num_lineups
count_df['lu_diff'] = 0




for iteration in tqdm(range(6)):
    for p, player in tqdm(enumerate(players)):
        #curr_df_len = len(lineup_df)

        #adjust_df_len = num_lineups - curr_df_len

        #target_own = pid_dict[p]['own']
        #target_lus = round(target_own * num_lineups) + adjust_df_len

        #actual_lus = count_df['count'].loc[count_df['name'] == player].values[0]

        #lu_diff = int(target_lus - actual_lus)
        
        lu_diff = lineup_difference(p, player)

        if lu_diff > 0:
            
            #Create list teams of 5 players without current player
            if p < 1:
                temp_player_ls = list(player_df['name'].unique())
                temp_player_ls.remove(player)
                
                if len(temp_player_ls) < 5:
                    print('error')
            else:
                temp_player_ls = list(count_df['name'].loc[count_df['lu_diff'] > 0].unique())
                temp_player_ls.remove(player)
                
            temp_5_man_lus = random_lineup_generator(temp_player_ls, 5, lu_diff, player)   

            #Add player to 5 man combos
            temp_lus = []
            for t, lu in enumerate(temp_5_man_lus):
                temp_lus.append(lu + (player,))

            temp_df = pd.DataFrame(temp_lus)

            #replace teams in lineup_df

            #select random indexs len num of lineups to replace
            removal_index = sorted(random.sample(range(num_lineups), lu_diff))

            new_df = lineup_df.loc[~lineup_df.index.isin(removal_index)]

            lineup_df = pd.concat([new_df, temp_df])
            lineup_df.reset_index(inplace = True, drop = True)


        if lu_diff < 0:
            lu_diff = abs(lu_diff)

            # Create 6 man teams to replace teams with player
            #temp_player_ls = list(player_df['name'].unique())
            if p < 1:
                temp_player_ls = list(player_df['name'].unique())
                temp_player_ls.remove(player)
                
                if len(temp_player_ls) < 6:
                    print('error')

            else:
                temp_player_ls = list(count_df['name'].loc[count_df['lu_diff'] > -200].unique())
                #temp_player_ls.remove(player)

            temp_6_man_lus = random_lineup_generator(temp_player_ls, 6, lu_diff, 'placeholder')
            temp_df = pd.DataFrame(temp_6_man_lus)

            #Remove existing lineups with player in them
            removal_index = lineup_df[lineup_df.apply(lambda x: x.str.contains(player, case = False).any(),
                                                      axis = 1)].sample(frac=1).head(lu_diff).index

            new_df = lineup_df.loc[~lineup_df.index.isin(removal_index)]

            lineup_df = pd.concat([new_df, temp_df])
            lineup_df.reset_index(inplace = True, drop = True)
            
            

        count_df = lineup_df.apply(pd.Series.value_counts)

        #Turn counts into percent
        count_df['count'] = count_df.sum(axis = 1)
        count_df.reset_index(inplace = True)
        count_df.rename(columns = {'index': 'name'}, inplace = True)
        count_df['owned'] = count_df['name'].apply(lambda x: return_ownership(x))
        count_df['target_lus'] = count_df['owned'] * num_lineups
        count_df['lu_diff'] = count_df['target_lus'] - count_df['count']
        
print(datetime.datetime.now() - start)       

  0%|          | 0/6 [00:00<?, ?it/s]
0it [00:00, ?it/s]
1it [00:02,  2.81s/it]
3it [00:05,  2.37s/it]
6it [00:08,  1.92s/it]
7it [00:10,  2.09s/it]
8it [00:13,  2.18s/it]
9it [00:13,  1.67s/it]
13it [00:13,  1.18s/it]
16it [00:16,  1.07s/it]
18it [00:18,  1.10s/it]
19it [00:20,  1.48s/it]
23it [00:23,  1.21s/it]
25it [00:25,  1.20s/it]
26it [00:27,  1.56s/it]
27it [00:30,  1.79s/it]
30it [00:32,  1.49s/it]
31it [00:35,  1.75s/it]
32it [00:37,  1.94s/it]
33it [00:39,  2.06s/it]
34it [00:42,  2.16s/it]
35it [00:44,  2.22s/it]
36it [00:46,  2.25s/it]
37it [00:49,  2.28s/it]
39it [00:51,  1.95s/it]
41it [00:53,  1.72s/it]
42it [00:56,  1.91s/it]
43it [00:58,  2.03s/it]
44it [01:00,  2.14s/it]
45it [01:03,  2.20s/it]
46it [01:05,  2.24s/it]
47it [01:07,  2.29s/it]
48it [01:10,  2.32s/it]
49it [01:12,  2.32s/it]
53it [01:15,  1.81s/it]
54it [01:17,  1.98s/it]
55it [01:19,  2.09s/it]
57it [01:22,  1.82s/it]
58it [01:24,  1.98s/it]
59it [01:26,  2.10s/it]
61it [01:29,  1.83s/it]
62it [01:31, 

0:20:18.859189


In [8]:
count_df.sort_values('lu_diff', ascending = False)

,name,0,1,2,3,4,5,count,owned,target_lus,lu_diff
94,Sergio Garcia,16.0,29.0,125.0,556.0,865.0,2650.0,4241.0,0.35,5250.0,1009.0
86,Rory McIlroy,41.0,52.0,257.0,670.0,895.0,1895.0,3810.0,0.30,4500.0,690.0
84,Rickie Fowler,32.0,51.0,330.0,800.0,773.0,1262.0,3248.0,0.25,3750.0,502.0
72,Matt Kuchar,35.0,265.0,747.0,729.0,375.0,618.0,2769.0,0.20,3000.0,231.0
57,Jordan Spieth,116.0,436.0,742.0,674.0,315.0,186.0,2469.0,0.18,2700.0,231.0
1,Adam Scott,2119.0,46.0,5.0,NaN,NaN,230.0,2400.0,0.17,2550.0,150.0
80,Paul Casey,11.0,128.0,434.0,745.0,449.0,134.0,1901.0,0.13,1950.0,49.0
20,Charley Hoffman,685.0,645.0,217.0,33.0,NaN,31.0,1611.0,0.11,1650.0,39.0
64,Kevin Na,41.0,182.0,348.0,287.0,129.0,176.0,1163.0,0.08,1200.0,37.0
102,Tommy Fleetwood,3.0,9.0,24.0,323.0,778.0,330.0,1467.0,0.10,1500.0,33.0


In [6]:
lineup_df

,0,1,2,3,4,5
0,Rory McIlroy,Matt Kuchar,Marc Leishman,Bernd Wiesberger,Kevin Na,Webb Simpson
1,Rory McIlroy,Justin Rose,Russell Henley,Ryan Moore,Graeme McDowell,Kevin Chappell
2,Rory McIlroy,Henrik Stenson,Matthew Fitzpatrick,Tony Finau,Francesco Molinari,Chez Reavie
3,Dustin Johnson,Jon Rahm,Graeme McDowell,Kevin Kisner,Bryson DeChambeau,Sean O'Hair
4,Rory McIlroy,Rickie Fowler,Ian Poulter,Kevin Chappell,Jonas Blixt,Jim Herman
5,Jordan Spieth,Phil Mickelson,Charl Schwartzel,Louis Oosthuizen,Hideto Tanihara,Chez Reavie
6,Hideki Matsuyama,Jason Day,J.B. Holmes,Ryan Moore,Kelly Kraft,Scott Brown
7,Rickie Fowler,Adam Scott,Marc Leishman,Alexander Noren,Francesco Molinari,Patrick Rodgers
8,Jon Rahm,Sergio Garcia,Russell Henley,Patrick Reed,Brendan Steele,Ryan Moore
9,Hideki Matsuyama,Jon Rahm,Daniel Berger,J.B. Holmes,Peter Uihlein,Kevin Na


In [8]:
team_ls

[('Brendan Steele',
  'Francesco Molinari',
  'Jim Furyk',
  'Justin Thomas',
  'Shane Lowry',
  'Zach Johnson'),
 ('Bill Haas',
  'Bryson DeChambeau',
  'Bud Cauley',
  'Hudson Swafford',
  'Russell Henley',
  'Webb Simpson'),
 ('Bill Haas',
  'David Lingmerth',
  'Paul Casey',
  'Phil Mickelson',
  'Russell Knox',
  'Webb Simpson'),
 ('Daniel Berger',
  'Graham DeLaet',
  'Grayson Murray',
  'Paul Casey',
  'Shane Lowry',
  'Webb Simpson'),
 ('Bryson DeChambeau',
  'Ian Poulter',
  'Marc Leishman',
  'Russell Henley',
  'Sung Kang',
  'Tony Finau'),
 ('Adam Hadwin',
  'Adam Scott',
  'Ian Poulter',
  'Kevin Na',
  'Patrick Cantlay',
  'Xander Schauffele'),
 ('Adam Scott',
  'Daniel Berger',
  'Justin Thomas',
  'Marc Leishman',
  'Phil Mickelson',
  'Xander Schauffele'),
 ('Charley Hoffman',
  'Grayson Murray',
  'Keegan Bradley',
  'Marc Leishman',
  'Pat Perez',
  'Russell Knox'),
 ('Adam Scott',
  'Charley Hoffman',
  'Daniel Berger',
  'Justin Thomas',
  'Paul Casey',
  'Phil Mic

In [29]:
player_df = pd.read_csv('/home/valesco/Datasets/PGA_Data/salary_folder/pga_champ_salaries.csv')
player_df['owned'].sum()

6.05999999999999

In [10]:
player_df['name'].values

array(['Jordan Spieth', 'Rory McIlroy', 'Dustin Johnson', 'Rickie Fowler',
       'Hideki Matsuyama', 'Jason Day', 'Jon Rahm', 'Henrik Stenson',
       'Sergio Garcia', 'Brooks Koepka', 'Justin Rose', 'Justin Thomas',
       'Matt Kuchar', 'Adam Scott', 'Phil Mickelson', 'Branden Grace',
       'Marc Leishman', 'Tommy Fleetwood', 'Alexander Noren', 'Paul Casey',
       'Daniel Berger', 'Charley Hoffman', 'Thomas Pieters',
       'Charl Schwartzel', 'Louis Oosthuizen', 'J.B. Holmes',
       'Matthew Fitzpatrick', 'Russell Henley', 'Patrick Reed',
       'Brendan Steele', 'Ryan Moore', 'Bubba Watson', 'Jamie Lovemark',
       'Kyle Stanley', 'Si Woo Kim', 'Rafael Cabrera-Bello',
       'Wesley Bryan', 'Ian Poulter', 'Emiliano Grillo', 'Tony Finau',
       'Xander Schauffele', 'Bernd Wiesberger', 'Peter Uihlein',
       'Padraig Harrington', 'Thorbjorn Olesen', 'Graeme McDowell',
       'Lucas Glover', 'Hudson Swafford', 'Kevin Chappell', 'Kevin Kisner',
       'Jimmy Walker', 'Brian Harm